In [37]:
from stmpy import Machine, Driver
import paho.mqtt.client as mqtt

import ipywidgets as widgets
import time
from IPython.display import display

from threading import Thread
import json

In [38]:
CHARGER_INDEX = 0

class Charger:

    def load_images(self):
        self.start = open("images/green_on.png", "rb").read()
        self.reserve = open("images/yellow_on.png", "rb").read()
        self.stop = open("images/red_on.png", "rb").read()

    # Send a message to start charging
    def on_button_start(self, b):
        self.stm.send("start_charging")

    def on_button_reserve(self, b):
        self.stm.send("reserve_charger")

    def on_button_stop(self, b):
        self.stm.send("stop_charger")

    def display(self):
        # Buttons
        self.button_start = widgets.Button(description="Start charging")
        self.button_start.on_click(self.on_button_start)

        self.button_reserve = widgets.Button(description="Reserve charger")
        self.button_reserve.on_click(self.on_button_reserve)

        self.button_stop = widgets.Button(description="Stop charger")
        self.button_stop.on_click(self.on_button_stop)
        style = {'description_width': 'initial'}

        # Text field to display states
        self.text1 = widgets.Text(value='', placeholder='', description='State:', disabled = True, style = style)
        self.text2 = widgets.Text(value='', placeholder='', description='Enter reservation code:', disabled = False, style = style)
        self.text3 = widgets.Text(value='', placeholder='', description='Normal chargingtime:', disabled = True, style = style)
        self.text4 = widgets.Text(value='', placeholder='', description='Overcharge time:', disabled = True, style = style)
        self.text5 = widgets.Text(value='', placeholder='', description='Total price:', disabled = True, style = style)
        display(widgets.HBox((self.text1, self.text2, self.text3, self.text4, self.text5)), widgets.HBox(( self.button_start, self.button_reserve, self.button_stop)))


    def __init__(self):
        self.load_images()
        self.display()

        self.client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION2)
        self.client.on_connect = self.on_connect
        self.client.on_message = self.on_message

        self.client.subscribe("reservations")
        self.client.subscribe("expirations")

        global chargingcode_test, electricity_price, price_hour
        #Price of electricity
        electricity_price = 0.46
        #Effect on the charger
        effect_of_charger = 22
        #Price for 1 hour charging
        price_hour = electricity_price * effect_of_charger
        chargingcode_test = "test"

    def on_connect(self, client, userdata, flags, rc, properties):
        if rc == 0:
            print("Successfully connected to MQTT broker")
            client.publish("debug", "MQTT er online:)")  # Publiser en testmelding etter vellykket tilkobling
        else:
            print("Failed to connect, return code %d\n", rc)

    def on_message(self, client, userdata, msg):
        print(msg)
        if msg.topic == "reservations":
            self.stm.send("reserve_charger")

        elif msg.topic == "expirations":
            self.stm.send("reservation_expired")
        

    def connect(self, host, port):
        self.client.connect(host, port)
        print("Connection with MQTT broker established")

        thread = Thread(target=self.client.loop_forever)
        thread.start()


    def func_available(self):
        self.text1.set_trait(name="value", value="Available")

    def func_reserved(self):
        self.text1.set_trait(name="value", value="Reserved")
        self.button_reserve.layout.visibility = 'hidden'

    def check_chargingcode(self):
        if self.text2.value == chargingcode_test:
            return 'charging'
        else:
            return 'reserved'

    def func_charging(self):
        self.text1.set_trait(name="value", value="Charging")
        self.client.publish("arrivals", json.dumps({"spot_position": CHARGER_INDEX, 'reservation_code': self.text2.value}))

    def func_charging_battery_full(self):
        self.text1.set_trait(name="value", value="Charging with full battery")

    def func_overcharge(self):
        self.text1.set_trait(name="value", value="Overcharging")

    def func_finish_charging(self):
        self.text1.set_trait(name="value", value="Finished charging")
        self.client.publish("departures", json.dumps({"spot_position": CHARGER_INDEX}))

    #Function to take the time charging with normal price
    def start_payment(self):
        global start_charging_timer
        start_charging_timer = time.time()
        return
    
    ## Fucntion to stop the time charging with normal price
    def stop_payment(self):
        global stop_charging_timer, total_charging_time, start_charging_timer
        stop_charging_timer = time.time()
        total_charging_time = stop_charging_timer - start_charging_timer
        self.text3.set_trait(name="value", value=str(total_charging_time))

        return
    
    # Function to start the time when overcharged
    def start_overchargepayment(self):
        global start_overcharge_timer
        start_overcharge_timer = time.time()
        #Simulate time spent in overcharge
        time.sleep(1)
        return
    
    #Function to stop the time when overcharged
    def stop_overchargepayment(self):
        global stop_overcharge_timer, total_overcharge_time, start_overcharge_timer
        stop_overcharge_timer = time.time()
        total_overcharge_time = stop_overcharge_timer - start_overcharge_timer
        self.text4.set_trait(name="value", value=str(total_overcharge_time))
        return
    
    # Function to lock the plug
    def lock_plug(self):
        return
    
    # Function to unlock the plug
    def unlock_plug(self):
        return
    
    # Function to send info about price of the charging session to the payment service
    def charge_customer(self):
        global total_charging_time, total_overcharge_time, electricity_price
        charging_price = total_charging_time * price_hour
        overcharge_price = total_overcharge_time * 60 * 5

        total_price = charging_price + overcharge_price
        self.text5.set_trait(name="value", value=str(total_price) + " nok")
        time.sleep(4)
        self.stm.send("charger_available")

        return

    # Function to start the power to charge the car
    def start_power(self):
        #Use wait to simulate charging time
        time.sleep(1)
        return
    
    # Function to stop the power to charge the car
    def stop_power(self):
        return

charger = Charger()
#Initial transition
t0 = {'source':'initial',
      'target':'available'}

#Transition from available to reserved
t1 = {'trigger':'reserve_charger',
      'source':'available',
      'target':'reserved'}


#Transition from availble to charging
t2 = {'trigger': 'start_charging',
      'source':'available',
      'target':'charging'}

#Transition from charging to finish_charging
t3 = {'trigger':'stop_charger',
      'source': 'charging',
      'target':'finish_charging'}

#Transition from charging to charging_battery_full
t4 = {'trigger':'t3',
      'source':'charging',
      'target':'charging_battery_full'}

#Transition from charging_battery_full to finish_charging
t5 = {'trigger':'stop_charger',
      'source':'charging_battery_full',
      'target':'finish_charging'}

#Transition from charging_battery_full to overcharge
t6 = {'trigger':'t2',
      'source':'charging_battery_full',
      'target':'overcharge'}

#Transition from overcharge to finish_charging
t7 = {'trigger':'stop_charger',
      'source':'overcharge',
      'target':'finish_charging'}

#Transition from finish_charging to available
t8 = {'trigger':'charger_available',
      'source':'finish_charging',
      'target':'available'}

#Transition from reserved to available
t9 = {'trigger':'t1',
      'source':'reserved',
      'target':'available'}

#Transition from reserved to charging
t10 = {'trigger':'start_charging',
       'source':'reserved',
       'function': charger.check_chargingcode,
       'effect':'stop_timer("t1")'}

# STATES

available = {'name':'available',
             'entry':'func_available' }

reserved = {'name':'reserved',
            'entry':'func_reserved; start_timer("t1", 10000)'}

charging = {'name':'charging',
            'entry':'func_charging; start_payment; lock_plug; start_power; start_timer("t3", 1000)'}

charging_battery_full = {'name':'charging_battery_full',
                         'entry':'func_charging_battery_full; start_timer("t2", 1000)'}

overcharge = {'name':'overcharge',
              'entry':'start_overchargepayment; func_overcharge; stop_payment'}

finish_charging = {'name':'finish_charging',
                   'entry':'func_finish_charging; stop_power; stop_overchargepayment; unlock_plug; charge_customer; '}



machine = Machine(name="charger", transitions=[t0,t1,t2,t3,t4,t5,t6,t7,t8,t9, t10], obj=charger, states=[available, reserved, charging, charging_battery_full, overcharge, finish_charging])
charger.stm = machine

driver = Driver()
driver.add_machine(machine)
driver.start()

charger.connect("broker.hivemq.com", 1883)

Connection with MQTT broker established


Successfully connected to MQTT broker
